In [9]:
import pandas as pd
import numpy as np
import json

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Embedding,Dropout
from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam


In [2]:
dataframe= pd.read_csv('comment__learn.csv',sep='\t')
list__comment = dataframe['comment'].fillna('').values.tolist()
list__skore=dataframe['skore'].values.tolist()


In [3]:
cutoff=int(len(dataframe)*0.98)
train_comment,test_comment=list__comment[:cutoff],list__comment[cutoff:]
train_skore, test_skore=list__skore[:cutoff],list__skore[cutoff:]

In [4]:
train_comment[1111]

'yıllardıt kullandığımız ürün çok kaliteli ve fiyat çok uygundu teşekkürler'

In [5]:
with open('tokenizer.json', encoding='utf-8') as json_data:
    jsonData = json.load(json_data)


In [6]:
def token(commentList):
    y_comments=[]
    for comment in commentList:
        y_comment=[]
        for word in comment.split():
            if(len(y_comment)<50 and word in jsonData):
                y_comment.append(jsonData[word])

        if(len(y_comment)<50):
            zeros=list(np.zeros(50-len(y_comment),dtype=int))
            y_comment=zeros+y_comment

        y_comments.append(y_comment)

    return np.array(y_comments,dtype=np.dtype(np.int32))

In [7]:
train_cluster=token(train_comment)
test_cluster=token(test_comment)
train_skore_array = np.array(train_skore)
test_skore_array = np.array(test_skore)

In [10]:
model = Sequential()
model.add(Embedding(input_dim=70000,
                    output_dim=50,
                    input_length=50,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(Dropout(0.2))  

model.add(GRU(units=8, return_sequences=True))
model.add(Dropout(0.2)) 

model.add(GRU(units=4))
model.add(Dropout(0.2))  

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

model.summary()
              

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(train_cluster, train_skore_array, epochs=5, batch_size=128)

Epoch 1/5
13115/13115 ━━━━━━━━━━━━━━━━━━━━ 778s 59ms/step - accuracy: 0.9698 - loss: 0.0993
Epoch 2/5
13115/13115 ━━━━━━━━━━━━━━━━━━━━ 809s 62ms/step - accuracy: 0.9844 - loss: 0.0483
Epoch 3/5
13115/13115 ━━━━━━━━━━━━━━━━━━━━ 818s 62ms/step - accuracy: 0.9881 - loss: 0.0382
Epoch 4/5
13115/13115 ━━━━━━━━━━━━━━━━━━━━ 806s 61ms/step - accuracy: 0.9907 - loss: 0.0315
Epoch 5/5
13115/13115 ━━━━━━━━━━━━━━━━━━━━ 783s 60ms/step - accuracy: 0.9920 - loss: 0.0275


In [12]:
model.evaluate(test_cluster,test_skore_array)

1071/1071 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9832 - loss: 0.0513


[0.05505126714706421, 0.9826610088348389]

In [13]:
model.save('text_first_model1.keras')
##model.save('text_first.h5')


In [16]:
text_41 = "Ürünü aldıktan sonra yaşadığım memnuniyetsizlik, hızlıca giderildi."
text_42 = "Ürün beklediğimden daha kötü çıktı, hayal kırıklığına uğradım."
text_43 = "Kullanma kılavuzu çok eksik, ürünü kullanmak zor."
text_44 = "Fiyatı yüksek ancak kalitesiz bir ürün, hayal kırıklığı."
text_45 = "Ürün ambalajı sağlam değil, kargo sırasında zarar görmüş."
text_46 = "Ürünü aldığımda eksik parçalar vardı, hala çözülemedi."
text_47 = "Ürün çok geç geldi, kargo süresi çok uzun."
text_48 = "Ürünü kullanmaya başladıktan sonra arızalandı, tamir edilmesi gerekiyor."
text_49 = "Fiyatı düşük olmasına rağmen kalitesiz bir ürün, beklediğim gibi değil."
text_50 = "Ürünün malzeme kalitesi beklediğimden düşük, hayal kırıklığı yaşadım."
texts = [
    text_45, text_42,text_41,text_43,text_44,text_46,text_47,text_48,text_49,
]


In [17]:
test_data=token(texts)
response=model.predict(test_data)
response

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step


array([[0.92076975],
       [0.52836347],
       [0.98649085],
       [0.99068177],
       [0.27647507],
       [0.6580956 ],
       [0.70787406],
       [0.00165258],
       [0.01893957]], dtype=float32)